In [1]:
import pandas as pd
import numpy as np

In [ ]:
price_2023 = pd.read_csv("C:/Users/kshit/Downloads/2023.csv")
price_2022 =  pd.read_csv("C:/Users/kshit/Downloads/2022.csv")
price_2024 =  pd.read_csv("C:/Users/kshit/Downloads/2024.csv")

In [ ]:
pricedf_dict = {'price_2022': price_2022, 'price_2023': price_2023, 'price_2024': price_2024}

# Dictionary to store the results
avg_price_results = {}

# Process each DataFrame
for name, df in pricedf_dict.items():
    
    # Convert `date` column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Extract year and month for grouping
    df['year_month'] = df['date'].dt.to_period('M')
    
    # Group by `district` and `year_month`, then calculate the average `modal_price`
    df_avg_price = df.groupby(['district', 'year_month']).agg(avg_modal_price=('modal_price', 'mean')).reset_index()
    
    # Convert `year_month` back to string format if desired
    df_avg_price['year_month'] = df_avg_price['year_month'].astype(str)
    
    # Store the result in the dictionary with the key as the DataFrame name
    avg_price_results[name] = df_avg_price

# Display the results
for year, df_avg_price in avg_price_results.items():
    print(f"Results for {year}:\n", df_avg_price, "\n")

In [ ]:
import pandas as pd

# Combine all years' average price data into a single DataFrame
all_avg_prices = pd.concat(avg_price_results.values())

# Calculate the overall average `modal_price` for each district across all years
district_avg_price = all_avg_prices.groupby('district')['avg_modal_price'].mean().reset_index()

# Calculate the median average price across all districts (or you could use the overall mean)
overall_median_price = district_avg_price['avg_modal_price'].median()

# Add a column to classify each district as "High-Demand" or "Low-Demand"
district_avg_price['demand_category'] = district_avg_price['avg_modal_price'].apply(
    lambda x: 'High-Demand' if x > overall_median_price else 'Low-Demand'
)

# Display the high-demand and low-demand classification
print(district_avg_price)
district_avg_price.to_excel("C:/Users/kshit/Downloads/potato_pricedata.xlsx", index=False)

In [2]:
bihar_cold_storage = pd.read_excel('C:/Users/kshit/Downloads/bihar_cold_storage.xlsx')
biharpotatodata = pd.read_excel('C:/Users/kshit/Downloads/biharpotatodata.xlsx')
potato_pricedata = pd.read_excel('C:/Users/kshit/Downloads/potato_pricedata.xlsx')
df_distances = pd.read_excel('C:/Users/kshit/Downloads/df_distances.xlsx')

In [3]:
potato_pricedata['Average Price per Quintal'] = potato_pricedata['avg_modal_price']
potato_pricedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   district                   39 non-null     object 
 1   avg_modal_price            39 non-null     float64
 2   demand_category            39 non-null     object 
 3   Average Price per Quintal  39 non-null     float64
dtypes: float64(2), object(2)
memory usage: 1.3+ KB


In [4]:
potato_pricedata['District'] = potato_pricedata['district']
potato_pricedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   district                   39 non-null     object 
 1   avg_modal_price            39 non-null     float64
 2   demand_category            39 non-null     object 
 3   Average Price per Quintal  39 non-null     float64
 4   District                   39 non-null     object 
dtypes: float64(2), object(3)
memory usage: 1.6+ KB


In [6]:
sufficiency_results = []

for idx, row in biharpotatodata.iterrows():
    district = row['District']
    production = row['Production (Tonnes)']
    required_storage = row['Production (Tonnes)']
    
    # Calculate total storage capacity for the district in bihar_cold_storage
    total_capacity = bihar_cold_storage.loc[bihar_cold_storage['District'] == district, 'Capacity'].sum()
    
    # Determine if storage is sufficient
    is_sufficient = total_capacity >= required_storage
    if is_sufficient:
        extra_storage_space = total_capacity - required_storage
        more_space_needed = 0
    else:
        extra_storage_space = 0
        more_space_needed = required_storage - total_capacity

    # Store result
    sufficiency_results.append({
        'District': district,
        'Production': production,
        'Required Storage for Potatoes': required_storage,
        'Total Cold Storage Capacity': total_capacity,
        'Is Sufficient': is_sufficient,
        'Extra Storage Space': extra_storage_space,
        'More Space Required': more_space_needed
    })

# Convert results to DataFrame
sufficiency_df = pd.DataFrame(sufficiency_results)

# Display the result
print(sufficiency_df)
surplusspace_df = sufficiency_df[sufficiency_df['Extra Storage Space'] > 0]
shortfallspace_df = sufficiency_df[sufficiency_df['More Space Required'] > 0]

              District  Production  Required Storage for Potatoes  \
0               ARARIA    27405.00                       27405.00   
1                ARWAL    13850.00                       13850.00   
2           AURANGABAD    14505.00                       14505.00   
3                BANKA    14905.00                       14905.00   
4            BEGUSARAI    57363.00                       57363.00   
5            BHAGALPUR     6549.00                        6549.00   
6              BHOJPUR    65772.00                       65772.00   
7                BUXAR    20128.00                       20128.00   
8            DARBHANGA    33256.00                       33256.00   
9                 GAYA    55463.00                       55463.00   
10           GOPALGANJ    28852.00                       28852.00   
11               JAMUI    13281.00                       13281.00   
12           JEHANABAD     5244.00                        5244.00   
13     KAIMUR (BHABUA)     4348.00

In [7]:
# Model 1

import pandas as pd
import pulp

def calculate_profit(distance, transport_amount, potato_price, transport_cost_per_km=1.5, price_per_quintal=10):
    """ Calculate profit for transporting potatoes. """
    return (((potato_price * price_per_quintal - transport_cost_per_km * price_per_quintal * distance) / price_per_quintal) * transport_amount)

def solve_transportation_problem(shortfallspace_df, surplusspace_df, df_distances, potato_pricedata, wastage_percentage=0.15):
    all_results = []

    for shortfall_district in shortfallspace_df['District']:
        # Define the LP problem for each shortfall district
        lp = pulp.LpProblem(f"Potato_Transportation_Problem_{shortfall_district}", pulp.LpMaximize)

        # Create decision variables: transport from shortfall to surplus district
        x = pulp.LpVariable.dicts(f"Transport_{shortfall_district}",
                                  [(i, j) for i in shortfallspace_df['District'] for j in surplusspace_df['District']],
                                  lowBound=0, cat='Continuous')

        # Objective function: Maximize total transportation amount
        objective = pulp.lpSum(x[i, j] for i in shortfallspace_df['District'] for j in surplusspace_df['District'])
        lp += objective, "Total_Transport_Amount"

        # Add constraints: Shortfall district demand after wastage consideration
        for i in shortfallspace_df['District']:
            adjusted_shortfall_demand = shortfallspace_df[shortfallspace_df['District'] == i]['More Space Required'].values[0] * (1 - wastage_percentage)
            lp += pulp.lpSum(x[i, j] for j in surplusspace_df['District']) <= adjusted_shortfall_demand, f"Shortfall_Produce_Constraint_{i}"

        # Add constraints: Surplus district capacity
        for j in surplusspace_df['District']:
            lp += pulp.lpSum(x[i, j] for i in shortfallspace_df['District']) <= surplusspace_df[surplusspace_df['District'] == j]['Extra Storage Space'].values[0], f"Surplus_Capacity_Constraint_{j}"

        # Solve the LP problem
        lp.solve()

        # If optimal solution found, collect the results
        if pulp.LpStatus[lp.status] == 'Optimal':
            transport_plan = []
            total_cold_storage_needed = {}

            for i in shortfallspace_df['District']:
                for j in surplusspace_df['District']:
                    transport_amount = pulp.value(x[i, j])
                    if transport_amount > 0:
                        distance = df_distances[(df_distances['From'] == i) & (df_distances['To'] == j)]['Distance_km'].values[0]
                        potato_price = potato_pricedata[potato_pricedata['District'] == j]['Average Price per Quintal'].values[0]
                        
                        # Calculate profit using the new function
                        profit_value = calculate_profit(distance, transport_amount, potato_price)
                        transport_plan.append({
                            'From District': i,
                            'To District': j,
                            'Transport Amount (Metric Tonnes)': transport_amount,
                            'Profit': profit_value
                        })

                    # Track cold storage space required for surplus districts
                    if j not in total_cold_storage_needed:
                        total_cold_storage_needed[j] = 0
                    total_cold_storage_needed[j] += transport_amount

            # Calculate extra cold storage needed for each surplus district
            cold_storage_plan = []
            for j in surplusspace_df['District']:
                current_storage = surplusspace_df[surplusspace_df['District'] == j]['Extra Storage Space'].values[0]
                required_storage = total_cold_storage_needed.get(j, 0)
                extra_storage_needed = max(0, required_storage - current_storage)
                cold_storage_plan.append({
                    'District': j,
                    'Current Storage (Metric Tonnes)': current_storage,
                    'Required Storage (Metric Tonnes)': required_storage,
                    'Extra Storage Needed (Metric Tonnes)': extra_storage_needed
                })

            # Convert transport and cold storage plans to DataFrames
            transport_plan_df = pd.DataFrame(transport_plan)
            cold_storage_df = pd.DataFrame(cold_storage_plan)

            # Merge transport and cold storage plans
            final_df = pd.merge(transport_plan_df, cold_storage_df, how='left', left_on='To District', right_on='District')
            final_df.drop('District', axis=1, inplace=True)  # Drop 'District' column after merge
            final_df = final_df.rename(columns={'To District': 'Surplus District'})

            # Append the results for this shortfall district
            all_results.append(final_df)

        else:
            print(f"No optimal solution found for {shortfall_district}.")

    return all_results

# Example usage
# Assuming your dataframes (shortfallspace_df, surplusspace_df, df_distances, and potato_pricedata) are defined
all_results = solve_transportation_problem(shortfallspace_df, surplusspace_df, df_distances, potato_pricedata)

# Output the results
for result in all_results:
    print(result)

c:\Users\kshit\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


         From District Surplus District  Transport Amount (Metric Tonnes)  \
0                ARWAL        BEGUSARAI                           9237.00   
1           AURANGABAD             GAYA                           1167.00   
2                BANKA          NALANDA                           1355.00   
3              BHOJPUR        JEHANABAD                            756.00   
4              BHOJPUR           ROHTAS                           1260.00   
5            DARBHANGA      MUZAFFARPUR                           8441.00   
6                JAMUI  KAIMUR (BHABUA)                           1252.00   
7              KATIHAR         VAISHALI                           8796.00   
8           KISHANGANJ           PURNIA                            670.95   
9            MADHEPURA        BHAGALPUR                          10594.90   
10           MADHEPURA  PURBI CHAMPARAN                           1823.00   
11           MADHUBANI           PURNIA                          22683.10   

In [9]:
percentage_space_utilisation = result['Transport Amount (Metric Tonnes)'].sum()/shortfallspace_df['More Space Required'].sum()
percentage_space_utilisation 

0.21431636022176734

In [10]:
Total_profit = result['Profit'].sum()
Total_profit

248103738.40928766

In [11]:
merged_df = pd.merge(result, shortfallspace_df, left_on='From District', right_on='District', how='left')

# Calculate the remaining extra produce
# Group by the 'From District' and sum the transport amounts (since a district can have multiple surplus districts)
total_transport_per_district = merged_df.groupby('From District')['Transport Amount (Metric Tonnes)'].sum().reset_index()

# Merge the total transport per district with the shortfallspace_df to calculate the remaining produce
remaining_produce_df = pd.merge(shortfallspace_df, total_transport_per_district, left_on='District', right_on='From District', how='left')

# Calculate the remaining extra produce
remaining_produce_df['Remaining Extra Produce'] = remaining_produce_df['More Space Required'] - remaining_produce_df['Transport Amount (Metric Tonnes)']
# Calculate the remaining extra produce
remaining_produce_df['Percentage Allocation'] = (remaining_produce_df['Transport Amount (Metric Tonnes)'] / remaining_produce_df['More Space Required']) * 100

# Calculate the percentage unallocated
remaining_produce_df['Percentage Unallocated'] = (remaining_produce_df['Remaining Extra Produce'] / remaining_produce_df['More Space Required']) * 100

# Display the result with the new columns
print(remaining_produce_df[['District', 'Remaining Extra Produce', 'Percentage Allocation', 'Percentage Unallocated']])

              District  Remaining Extra Produce  Percentage Allocation  \
0               ARARIA                      NaN                    NaN   
1                ARWAL                  4613.00              66.693141   
2           AURANGABAD                 11938.00               8.904998   
3                BANKA                 13550.00               9.090909   
4              BHOJPUR                 47256.00               4.091573   
5            DARBHANGA                 24115.00              25.927632   
6                JAMUI                 12029.00               9.427001   
7              KATIHAR                 25946.00              25.318059   
8             KHAGARIA                      NaN                    NaN   
9           KISHANGANJ                 23037.05               2.830057   
10           MADHEPURA                 34483.10              26.476834   
11           MADHUBANI                  4002.90              85.000000   
12              MUNGER                

In [12]:
remaining_produce_df = remaining_produce_df[['District',  'Percentage Allocation', 'Percentage Unallocated']]

In [14]:
remaining_produce_df.to_csv("C:/Users/kshit/Downloads/scm project/scm project shared/remaining_produce_df.csv",index = False)

In [15]:
result.to_csv("C:/Users/kshit/Downloads/scm project/scm project shared/result.csv",index = False)

In [16]:
import pandas as pd
import pulp

def calculate_profit(distance, transport_amount, potato_price, transport_cost_per_km=1.5, price_per_quintal=10):
    """
    Calculate profit for transporting potatoes.
    """
    return ((potato_price * price_per_quintal - transport_cost_per_km * price_per_quintal * distance) / price_per_quintal) * transport_amount

def solve_transportation_problem(shortfallspace_df, surplusspace_df, df_distances, potato_pricedata, wastage_percentage=0.15,
                                 shortfall_relaxation_percentage=0.1, surplus_capacity_relaxation_percentage=0.1):
    """
    Solve the transportation problem by maximizing profit with relaxed constraints.
    """
    all_results = []

    for shortfall_district in shortfallspace_df['District']:
        # Define the LP problem for each shortfall district
        lp = pulp.LpProblem(f"Potato_Transportation_Problem_{shortfall_district}", pulp.LpMaximize)

        # Create decision variables: transport from shortfall to surplus district
        x = pulp.LpVariable.dicts(f"Transport_{shortfall_district}",
                                  [(i, j) for i in shortfallspace_df['District'] for j in surplusspace_df['District']],
                                  lowBound=0, cat='Continuous')

        # Objective function: Maximize total profit
        objective = pulp.lpSum(calculate_profit(
            df_distances[(df_distances['From'] == i) & (df_distances['To'] == j)]['Distance_km'].values[0],
            x[i, j],
            potato_pricedata[potato_pricedata['District'] == j]['Average Price per Quintal'].values[0]
        ) for i in shortfallspace_df['District'] for j in surplusspace_df['District'])
        lp += objective, "Total_Profit"

        # Add constraints: Shortfall district demand after wastage consideration with relaxation
        for i in shortfallspace_df['District']:
            adjusted_shortfall_demand = shortfallspace_df[shortfallspace_df['District'] == i]['More Space Required'].values[0] * (1 - wastage_percentage)
            relaxed_shortfall_demand = adjusted_shortfall_demand * (1 + shortfall_relaxation_percentage)
            lp += pulp.lpSum(x[i, j] for j in surplusspace_df['District']) <= relaxed_shortfall_demand, f"Relaxed_Shortfall_Produce_Constraint_{i}"

        # Add constraints: Surplus district capacity with relaxation
        for j in surplusspace_df['District']:
            current_capacity = surplusspace_df[surplusspace_df['District'] == j]['Extra Storage Space'].values[0]
            relaxed_capacity = current_capacity * (1 + surplus_capacity_relaxation_percentage)
            lp += pulp.lpSum(x[i, j] for i in shortfallspace_df['District']) <= relaxed_capacity, f"Relaxed_Surplus_Capacity_Constraint_{j}"

        # Solve the LP problem
        lp.solve()

        # If optimal solution found, collect the results
        if pulp.LpStatus[lp.status] == 'Optimal':
            transport_plan = []
            total_cold_storage_needed = {}

            for i in shortfallspace_df['District']:
                for j in surplusspace_df['District']:
                    transport_amount = pulp.value(x[i, j])
                    if transport_amount > 0:
                        distance = df_distances[(df_distances['From'] == i) & (df_distances['To'] == j)]['Distance_km'].values[0]
                        potato_price = potato_pricedata[potato_pricedata['District'] == j]['Average Price per Quintal'].values[0]
                        
                        # Calculate profit using the new function
                        profit_value = calculate_profit(distance, transport_amount, potato_price)
                        transport_plan.append({
                            'From District': i,
                            'To District': j,
                            'Transport Amount (Metric Tonnes)': transport_amount,
                            'Profit': profit_value
                        })

                    # Track cold storage space required for surplus districts
                    if j not in total_cold_storage_needed:
                        total_cold_storage_needed[j] = 0
                    total_cold_storage_needed[j] += transport_amount

            # Calculate extra cold storage needed for each surplus district
            cold_storage_plan = []
            for j in surplusspace_df['District']:
                current_storage = surplusspace_df[surplusspace_df['District'] == j]['Extra Storage Space'].values[0]
                required_storage = total_cold_storage_needed.get(j, 0)
                extra_storage_needed = max(0, required_storage - current_storage)
                cold_storage_plan.append({
                    'District': j,
                    'Current Storage (Metric Tonnes)': current_storage,
                    'Required Storage (Metric Tonnes)': required_storage,
                    'Extra Storage Needed (Metric Tonnes)': extra_storage_needed
                })

            # Convert transport and cold storage plans to DataFrames
            transport_plan_df = pd.DataFrame(transport_plan)
            cold_storage_df = pd.DataFrame(cold_storage_plan)

            # Merge transport and cold storage plans
            final_df = pd.merge(transport_plan_df, cold_storage_df, how='left', left_on='To District', right_on='District')
            final_df.drop('District', axis=1, inplace=True)  # Drop 'District' column after merge
            final_df = final_df.rename(columns={'To District': 'Surplus District'})

            # Append the results for this shortfall district
            all_results.append(final_df)

        else:
            print(f"No optimal solution found for {shortfall_district}.")

    return all_results


# Assuming you already have the following dataframes: shortfallspace_df, surplusspace_df, df_distances, potato_pricedata
results = solve_transportation_problem(shortfallspace_df, surplusspace_df, df_distances, potato_pricedata, wastage_percentage=0.15, 
                                       shortfall_relaxation_percentage=0.1, surplus_capacity_relaxation_percentage=0.1)

# Merging all result DataFrames into one
final_results_df = pd.concat(results, ignore_index=True)

# Display the final result
print(final_results_df)


c:\Users\kshit\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


    From District Surplus District  Transport Amount (Metric Tonnes)  \
0           ARWAL        JEHANABAD                            831.60   
1           ARWAL            PATNA                          12118.15   
2      AURANGABAD             GAYA                            713.46   
3      AURANGABAD  KAIMUR (BHABUA)                           1377.20   
4      AURANGABAD           ROHTAS                           1386.00   
..            ...              ...                               ...   
501       SHEOHAR      MUZAFFARPUR                           2372.37   
502       SHEOHAR  PURBI CHAMPARAN                           2005.30   
503     SITAMARHI      MUZAFFARPUR                           6912.73   
504         SIWAN        GOPALGANJ                           5827.80   
505         SIWAN            SARAN                          26194.08   

           Profit  Current Storage (Metric Tonnes)  \
0    2.118368e+06                            756.0   
1    2.361284e+07          

In [18]:
final_results_df = final_results_df.drop_duplicates()

In [24]:
final_results_df.drop(['Extra Storage Needed (Metric Tonnes)'],axis = 1,inplace = True)

In [21]:
Model2profit = final_results_df['Profit'].sum()
Model2profit

297888748.9566718

In [23]:
percentagespaceutilization = (final_results_df['Transport Amount (Metric Tonnes)'].sum()/shortfallspace_df['More Space Required'].sum())*100
percentagespaceutilization

23.57479962439441

In [26]:
final_results_df.to_csv("C:/Users/kshit/Downloads/scm project/scm project shared/final_result_df.csv",index = False)

In [27]:
district_summary = []

# Loop through each unique district
for district in final_results_df['From District'].unique():
    # Sum the transport amounts for each 'From District'
    sum_transports = final_results_df[final_results_df['From District'] == district]['Transport Amount (Metric Tonnes)'].sum()

    # Get the corresponding required storage for the district from shortfallspace_df
    required_storage = shortfallspace_df[shortfallspace_df['District'] == district]['Required Storage for Potatoes'].values[0]

    # Calculate the remaining unallocated space
    remaining_unallocated = required_storage - sum_transports

    # Calculate the percentage of unallocated space
    percent_unallocated = (remaining_unallocated / required_storage) * 100

    # Calculate the percentage of allocated space
    percent_allocated = 100 - percent_unallocated

    # Append the results as a dictionary to the list
    district_summary.append({
        'District': district,
        'Unallocated Percentage (%)': percent_unallocated,
        'Allocated Percentage (%)': percent_allocated
    })

# Convert the list of dictionaries into a DataFrame
district_summary_df = pd.DataFrame(district_summary)

# Display the DataFrame
print(district_summary_df)



      District  Unallocated Percentage (%)  Allocated Percentage (%)
0        ARWAL                    6.500000                 93.500000
1   AURANGABAD                   76.031300                 23.968700
2      BHOJPUR                   29.956030                 70.043970
3      KATIHAR                   44.454007                 55.545993
4     KHAGARIA                   76.994256                 23.005744
5    MADHEPURA                   71.144027                 28.855973
6       MUNGER                   78.492376                 21.507624
7       NAWADA                   78.557450                 21.442550
8   SAMASTIPUR                   90.463947                  9.536053
9   SHEIKHPURA                   94.707547                  5.292453
10     SHEOHAR                    6.500000                 93.500000
11   SITAMARHI                   91.041625                  8.958375
12       SIWAN                   36.962321                 63.037679


In [28]:
district_summary_df.to_csv("C:/Users/kshit/Downloads/scm project/scm project shared/district_summary.csv",index = False)